In [1]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
load_dotenv()

embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = Chroma(
    collection_name="others",  # 원하는 store로 변경
    persist_directory="crawling/vector_store/others",
    embedding_function=embedding_model
)
retriever = vector_store.as_retriever(search_kwargs={'k': 5})

query = "비타민 D와 운동선수 건강의 관계는?"
docs = retriever.get_relevant_documents(query)
print(f"검색된 문서 개수: {len(docs)}")
for i, doc in enumerate(docs):
    print(f"--- 검색 결과 {i+1} ---")
    print(doc.page_content if hasattr(doc, 'page_content') else doc)
    print()

C:\Users\Playdata\AppData\Local\Temp\ipykernel_21188\202686466.py:15: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


검색된 문서 개수: 5
--- 검색 결과 1 ---
Vitamin D [25(OH)D] is a key nutrient, although its level is often low in the general population. To investigate the relationship between vitamin D levels and muscle performance, and to analyze how vitamin D changes during a 16‐week competitive season and its relationship to the performance tests performed. Participant characteristics: age 25.1 ± 4.7 years; height 1.8 ± 0.1 m, and body mass 73.9 ± 15.4 kg. Vitamin D levels (ng/mL) were at T1 (September): 33.7 ± 14.7 (n= 23), and at T2 (January): 26.1 ± 7.3 (n= 23). Over 16 weeks of competition, participants' blood was analyzed to determine their vitamin D levels. Their athletic abilities were evaluated through various tests: vertical jumps (standing jump and countermovement jump); 20‐m sprint without direction changes; and intermittent endurance test; the vitamin D level decreased from T1 to T2 by −22. 40% [p< 0.05] but performance improved in all tests performed (SJ: 4.57%; CMJ: 6.94%; VO2max: 4.99% [p< 0.

In [2]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

prompt_template = PromptTemplate(
    template="""# Instruction:
당신은 정확한 정보 제공을 우선시하는 인공지능 어시스턴트입니다.
주어진 Context에 포함된 정보만 사용해서 질문에 답변하세요.
Context에 질문에 대한 명확한 정보가 있는 경우 그 내용을 바탕으로 답변하세요.
Context에 질문에 대한 명확한 정보없을 경우 "정보가 부족해서 답을 알 수 없습니다." 라고 대답합니다.

# Context:
{context}

# 질문:
{query}
"""
)

context = "\n\n".join([doc.page_content if hasattr(doc, 'page_content') else doc for doc in docs])
prompt = prompt_template.format(context=context, query=query)
response = llm.invoke(prompt)
print("LLM 답변:")
print(response.content)

LLM 답변:
비타민 D는 운동선수의 건강에 중요한 역할을 합니다. 구체적으로, 비타민 D는 칼슘 흡수를 도와 뼈 건강을 유지하고 스트레스 골절을 예방하며, 근육 기능을 촉진하여 근력 약화 및 부상을 방지합니다. 또한 호흡기 감염을 감소시키는 데에도 도움이 됩니다. 연구에 따르면 비타민 D 수치가 낮은 운동선수는 근골격계 증상이나 부상 발생 위험이 높아 비타민 D의 검진과 보충이 중요합니다. 일부 연구에서는 비타민 D 대사체 수준이 부상 발생과 관련이 있을 수 있다고 보고하였으나, 비타민 D 보충이 근기능 및 운동으로 인한 근육 손상에 미치는 영향에 대해서는 상반된 결과가 있습니다. 다만, 비타민 D 보충은 스트레스 골절 발생률을 줄이는 데 일관된 이점이 있음이 입증되었습니다. 또한, 비타민 D는 마그네슘과 밀접하게 연결되어 있어, 비타민 D가 마그네슘 흡수를 돕고 마그네슘은 비타민 D의 합성, 운반, 활성화에 중요합니다. 따라서 비타민 D는 운동선수의 근골격계 건강과 부상 예방에 중요한 영양소로서 역할을 합니다.


In [3]:
# context는 위에서 추출한 것 중 하나 사용
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

eval_llm = ChatOpenAI(model="gpt-4.1")
eval_prompt = PromptTemplate(
    template="""다음 [Context]를 바탕으로 질문-정답 쌍을 3개 생성하세요.
질문은 반드시 context에 있는 정보만 바탕으로 하세요.

#Context:
{context}
"""
)
eval_prompt_str = eval_prompt.format(context=context)
eval_response = eval_llm.invoke(eval_prompt_str)
print("질문-정답 쌍 생성 결과:")
print(eval_response.content)

질문-정답 쌍 생성 결과:
1. 질문: 16주간의 대회 시즌 동안 연구 참가자들의 비타민 D 수치는 어떻게 변화했으며, 이와 동시에 신체 수행 능력에는 어떤 변화가 있었습니까?
정답: 비타민 D 수치는 T1(9월)에서 33.7 ± 14.7 ng/mL에서 T2(1월) 26.1 ± 7.3 ng/mL로 22.40% 감소했지만, 신체 수행 능력은 모든 테스트에서 향상되었다(SJ: 4.57%, CMJ: 6.94%, VO2max: 4.99%, 20m 달리기: −1.83%).

2. 질문: 비타민 D와 마그네슘이 운동선수의 건강과 운동 수행에 각각 어떤 역할을 하는가?
정답: 비타민 D는 뼈 건강과 근기능, 호흡기 감염 감소에 중요하며, 마그네슘은 근기능, ATP 생성, 전해질 균형, 뼈 강도, 심혈관 건강에 중요하다.

3. 질문: 운동선수에서 비타민 D와 마그네슘이 서로 어떤 방식으로 상호작용하며, 이로 인해 왜 두 영양소가 모두 중요한가?
정답: 비타민 D는 마그네슘 흡수를 돕고, 마그네슘은 비타민 D의 합성, 운반, 활성화에 필요하므로 두 영양소가 서로의 대사와 기능에 밀접하게 연관되어 운동선수에게 모두 중요하다.


In [4]:
from ragas import EvaluationDataset, evaluate
from ragas.metrics import (
    LLMContextRecall, Faithfulness, LLMContextPrecisionWithReference, AnswerRelevancy
)
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
import pandas as pd

# 예시 데이터프레임 생성
eval_df = pd.DataFrame([{
    "user_input": query,
    "reference": "context에서 정답을 직접 추출하여 입력",
    "qa_context": context,
    "retrieved_contexts": [context],
    "response": response.content
}])

eval_dataset = EvaluationDataset.from_pandas(eval_df)
eval_llm = LangchainLLMWrapper(llm)
eval_embedding = LangchainEmbeddingsWrapper(embedding_model)
metrics = [
    LLMContextPrecisionWithReference(llm=eval_llm),
    LLMContextRecall(llm=eval_llm),
    Faithfulness(llm=eval_llm),
    AnswerRelevancy(llm=eval_llm, embeddings=eval_embedding)
]
eval_result = evaluate(dataset=eval_dataset, metrics=metrics)
print("RAGAS 평가 결과:")
print(eval_result.to_pandas())

Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

RAGAS 평가 결과:
             user_input                                 retrieved_contexts  \
0  비타민 D와 운동선수 건강의 관계는?  [Vitamin D [25(OH)D] is a key nutrient, althou...   

                                            response  \
0  비타민 D는 운동선수의 건강에 중요한 역할을 합니다. 구체적으로, 비타민 D는 칼슘...   

                  reference  llm_context_precision_with_reference  \
0  context에서 정답을 직접 추출하여 입력                                   1.0   

   context_recall  faithfulness  answer_relevancy  
0             0.0           1.0          0.677563  
